In [2]:
import sys
import pandas as pd
import json
import numpy as np
import torch
from torch import Tensor
import pickle
from tqdm.auto import tqdm

from matplotlib import pyplot as plt

sys.path.append("../")
from src.log import myLogger
from src.repository.data_repository import DataRepository
from src.checkpoint.checkpoint import Checkpoint
from src.metrics.jaccard import jaccard, calc_jaccard_mean

pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 600)

%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

In [3]:
logger = myLogger("../logs/temp.log", exp_id="e000", wdb_prj_id="temp", exp_config=None, use_wdb=False) 
dr = DataRepository(logger=logger, local_root_path="..")

2021-11-15 16:55:02,777 log.py               31   [INFO] [__init__] skip wandb init 


In [3]:
import pickle

train_prep_df = dr.load_preprocessed_df(
          dataset_name="train",                                                                                                                                                                                                                                   
          class_name="BaselineKernelPreprocessorV1",
          tokenizer_name="XLMRobertaTokenizerFast",
          max_length=400,
          pad_on_right=True,
          stride=135,
          split=False,
          lstrip=False,
          use_language_as_question=False,
          add_overflowing_batch_id=False,
    )

# mlqa_hindi_prep_df = dr.load_preprocessed_df(
#           dataset_name="mlqa_hindi",                                                                                                                                                                                                                                   
#           class_name="BaselineKernelPreprocessorV1",
#           tokenizer_name="XLMRobertaTokenizerFast",
#           max_length=400,
#           pad_on_right=True,
#           stride=135,
#           split=False,
#           lstrip=False,
#           use_language_as_question=False,
#           add_overflowing_batch_id=False,
#     )
# 
# xquad_prep_df = dr.load_preprocessed_df(
#           dataset_name="xquad",                                                                                                                                                                                                                                   
#           class_name="BaselineKernelPreprocessorV1",
#           tokenizer_name="XLMRobertaTokenizerFast",
#           max_length=400,
#           pad_on_right=True,
#           stride=135,
#           split=False,
#           lstrip=False,
#           use_language_as_question=False,
#           add_overflowing_batch_id=False,
#     )
# 
# prep_df = pd.concat([train_prep_df, mlqa_hindi_prep_df, xquad_prep_df], axis=0).reset_index(drop=True)
prep_df = train_prep_df
prep_df

2021-11-15 15:54:57,957 data_repository.py   262  [INFO] [load_preprocessed_df] loading data/preprocessed/train_BaselineKernelPreprocessorV1_XLMRobertaTokenizerFast_400_True_135_False_False_False_False.pkl ... 
2021-11-15 15:55:00,934 data_repository.py   269  [INFO] [load_preprocessed_df] done. 
2021-11-15 15:55:00,935 data_repository.py   262  [INFO] [load_preprocessed_df] loading data/preprocessed/mlqa_hindi_BaselineKernelPreprocessorV1_XLMRobertaTokenizerFast_400_True_135_False_False_False_False.pkl ... 
2021-11-15 15:55:02,514 data_repository.py   269  [INFO] [load_preprocessed_df] done. 
2021-11-15 15:55:02,515 data_repository.py   262  [INFO] [load_preprocessed_df] loading data/preprocessed/xquad_BaselineKernelPreprocessorV1_XLMRobertaTokenizerFast_400_True_135_False_False_False_False.pkl ... 
2021-11-15 15:55:02,713 data_repository.py   269  [INFO] [load_preprocessed_df] done. 


,id,context,question,answer_text,answer_start,language,top20_context,answer_text_count,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,part_answer_text_count,start_position,end_position,segmentation_position,is_contain_answer_text
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,-1,1,27,27,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
2,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
3,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",3,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
4,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",4,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22321,xquad_1185,विद्युत आवेश के परिवर्तन की समय दर के रूप में ...,इलेक्ट्रोस्टैटिक और चुंबकीय बल के योग के रूप क...,इलेक्ट्रोस्टैटिक बल,328,hindi,विद्युत आवेश के परिव,1,"[0, 234186, 2284, 17433, 3045, 71683, 871, 369...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,-1,1,116,121,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
22322,xquad_1186,उस आयतन के लिए प्रासंगिक क्रॉस-सेक्शनल क्षेत्र...,संरचनाओं में तनाव का कारण क्या बनता है?,तनाव टेंसर,343,hindi,उस आयतन के लिए प्रास,1,"[0, 120018, 11846, 421, 129558, 641, 6701, 600...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,-1,1,128,131,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
22323,xquad_1187,उस आयतन के लिए प्रासंगिक क्रॉस-सेक्शनल क्षेत्र...,किसी वस्तु के आयतन में क्रॉस सेक्शन क्षेत्र की...,दबाव की शर्तें,118,hindi,उस आयतन के लिए प्रास,1,"[0, 12820, 89773, 287, 34889, 41420, 421, 4761...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4,

#### load values

In [4]:
import gc
from collections import defaultdict
from tqdm.auto import tqdm

target_exp_ids = ["e048", "e049", "e051", "e053", "e054", "e073", "e074"]

prep_dfs = {}

for target_exp_id in tqdm(target_exp_ids):
    checkpoint_info = defaultdict(list)

    best_checkpoints = dr.best_checkpoint_filepaths(target_exp_id)
    for best_checkpoint in tqdm(best_checkpoints):
        exp_fold_checkpoint = dr.load_checkpoint_from_filepath(filepath_from_root=best_checkpoint, load_from_gcs=True, rm_local_after_load=True) 
        del exp_fold_checkpoint.model_state_dict
        del exp_fold_checkpoint.scheduler_state_dict
        del exp_fold_checkpoint.optimizer_state_dict
        gc.collect()

        checkpoint_info["val_ids"].extend(exp_fold_checkpoint.val_ids)
        checkpoint_info["val_start_logits"].extend(exp_fold_checkpoint.val_start_logits)
        checkpoint_info["val_end_logits"].extend(exp_fold_checkpoint.val_end_logits)
        del exp_fold_checkpoint
        gc.collect()

    val_start_logits = [np.asarray(logit) for logit in checkpoint_info["val_start_logits"]]
    val_end_logits = [np.asarray(logit) for logit in checkpoint_info["val_end_logits"]]    
    
    checkpoint_df = pd.DataFrame()
    checkpoint_df["id"] = checkpoint_info["val_ids"]
    checkpoint_df["start_logits"] = val_start_logits
    checkpoint_df["end_logits"] = val_end_logits

    checkpoint_df["overflowing_batch_id"] = None
    bef_id = ""
    overflowing_batch_id = 0
    for i, row in checkpoint_df.iterrows():
        if str(row["id"]) != bef_id:
            overflowing_batch_id = 0
        checkpoint_df.loc[i, "overflowing_batch_id"] = overflowing_batch_id
        bef_id = row["id"]
        overflowing_batch_id += 1    

    prep_dfs[target_exp_id]= prep_df.merge(checkpoint_df, on=["id", "overflowing_batch_id"], how="left")        

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

2021-11-15 15:55:03,332 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e048/best_checkpoint/0_0_0.7522_0.6789.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e048/best_checkpoint/0_0_0.7522_0.6789.pkl 
2021-11-15 15:55:55,740 repository.py        176  [INFO] [__download_from_gcs] download done. 
2021-11-15 15:56:18,942 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e048/best_checkpoint/2_2_0.8895_0.7167.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e048/best_checkpoint/2_2_0.8895_0.7167.pkl 
2021-11-15 15:57:13,189 repository.py        176  [INFO] [__download_from_gcs] download done. 
2021-11-15 15:57:20,479 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e048/best_checkpoint/3_1_0.7929_0.6597.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e048/best_checkpoint/3_1_0.7929_0.6597.pkl 
2021-11-15 15:58:08,342 repository.py        176  [INFO] [__download_from_gcs] download

  0%|          | 0/5 [00:00<?, ?it/s]

2021-11-15 15:59:15,579 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e049/best_checkpoint/0_0_2.0046_0.6503.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e049/best_checkpoint/0_0_2.0046_0.6503.pkl 
2021-11-15 16:00:04,996 repository.py        176  [INFO] [__download_from_gcs] download done. 
2021-11-15 16:00:11,539 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e049/best_checkpoint/1_0_2.0477_0.7238.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e049/best_checkpoint/1_0_2.0477_0.7238.pkl 
2021-11-15 16:01:03,330 repository.py        176  [INFO] [__download_from_gcs] download done. 
2021-11-15 16:01:10,590 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e049/best_checkpoint/2_1_2.1020_0.6980.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e049/best_checkpoint/2_1_2.1020_0.6980.pkl 
2021-11-15 16:02:04,249 repository.py        176  [INFO] [__download_from_gcs] download

...

2021-11-15 16:13:53,875 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e054/best_checkpoint/0_1_0.6810_0.6664.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e054/best_checkpoint/0_1_0.6810_0.6664.pkl 
2021-11-15 16:14:42,170 repository.py        176  [INFO] [__download_from_gcs] download done. 
2021-11-15 16:14:49,033 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e054/best_checkpoint/1_3_0.8346_0.7394.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e054/best_checkpoint/1_3_0.8346_0.7394.pkl 
2021-11-15 16:15:38,762 repository.py        176  [INFO] [__download_from_gcs] download done. 
2021-11-15 16:15:45,829 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e054/best_checkpoint/2_1_0.8445_0.7087.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e054/best_checkpoint/2_1_0.8445_0.7087.pkl 
2021-11-15 16:16:40,898 repository.py        176  [INFO] [__download_from_gcs] download

  0%|          | 0/5 [00:00<?, ?it/s]

2021-11-15 16:20:29,454 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e073/best_checkpoint/0_2_14.0360_0.6704.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e073/best_checkpoint/0_2_14.0360_0.6704.pkl 
2021-11-15 16:21:23,460 repository.py        176  [INFO] [__download_from_gcs] download done. 
2021-11-15 16:21:30,535 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e073/best_checkpoint/1_3_16.3359_0.6945.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e073/best_checkpoint/1_3_16.3359_0.6945.pkl 
2021-11-15 16:22:21,838 repository.py        176  [INFO] [__download_from_gcs] download done. 
2021-11-15 16:22:29,621 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e073/best_checkpoint/2_1_11.6360_0.6879.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e073/best_checkpoint/2_1_11.6360_0.6879.pkl 
2021-11-15 16:23:23,602 repository.py        176  [INFO] [__download_from_gcs] do

  0%|          | 0/5 [00:00<?, ?it/s]

2021-11-15 16:28:57,313 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e074/best_checkpoint/0_1_1.0233_0.6825.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e074/best_checkpoint/0_1_1.0233_0.6825.pkl 
2021-11-15 16:29:54,388 repository.py        176  [INFO] [__download_from_gcs] download done. 
2021-11-15 16:30:01,384 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e074/best_checkpoint/1_0_1.2723_0.7165.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e074/best_checkpoint/1_0_1.2723_0.7165.pkl 
2021-11-15 16:30:51,829 repository.py        176  [INFO] [__download_from_gcs] download done. 
2021-11-15 16:30:59,344 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e074/best_checkpoint/2_0_0.9045_0.7324.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e074/best_checkpoint/2_0_0.9045_0.7324.pkl 
2021-11-15 16:31:48,954 repository.py        176  [INFO] [__download_from_gcs] download

In [6]:
import pickle

with open("../data/dataset/optuna_ens/prep_dfs.pkl", "wb") as fout:
    pickle.dump(prep_dfs, fout)

## optuna ensemble

In [4]:
import pickle

with open("../data/dataset/optuna_ens/prep_dfs.pkl", "rb") as fin:
    prep_dfs = pickle.load(fin)

In [11]:
for k in prep_dfs:
    prep_dfs[k] = prep_dfs[k][prep_dfs[k].start_logits.notnull()]

In [5]:
optuna_exp_ids = ["e048", "e049", "e051", "e053", "e054", "e074"]

In [13]:
from src.postprocessor.factory import PostprocessorFactory

postprocessor = PostprocessorFactory(
    **{
        "postprocessor_type": "baseline_kernel",
        "n_best_size": 50,
        "max_answer_length": 30,
        "use_chars_length": False,
        "text_postprocess": "mypospro_ver1",
        "use_multiprocess": True,
    },
    logger=logger
).create()

In [16]:
def objective(trial):
    samples = [1./len(optuna_exp_ids)]*len(optuna_exp_ids)
    for i in range(len(optuna_exp_ids)):
        samples[i] = trial.suggest_uniform(optuna_exp_ids[i], 0, 1)
    
    w = [samples[i]/sum(samples) for i in range(len(samples))]
    
    ids = prep_dfs[optuna_exp_ids[0]]["id"]
    contexts = prep_dfs[optuna_exp_ids[0]]["context"]
    answer_texts = prep_dfs[optuna_exp_ids[0]]["answer_text"]
    offset_mappings = prep_dfs[optuna_exp_ids[0]]["offset_mapping"]
    
    start_logits = w[0] * prep_dfs[optuna_exp_ids[0]]["start_logits"]
    end_logits = w[0] * prep_dfs[optuna_exp_ids[0]]["end_logits"]
    for i in range(1, len(optuna_exp_ids)):
        start_logits += w[i] * prep_dfs[optuna_exp_ids[i]]["start_logits"]
        end_logits += w[i] * prep_dfs[optuna_exp_ids[i]]["end_logits"]

    pospro_ids, pospro_answer_texts, pospro_answer_preds = postprocessor(
        ids=ids,
        contexts=contexts,
        answer_texts=answer_texts,
        offset_mappings=offset_mappings,
        start_logits=start_logits,
        end_logits=end_logits,
    )
    val_jaccard = calc_jaccard_mean(
        text_trues=pospro_answer_texts, text_preds=pospro_answer_preds
    )
    return val_jaccard

In [17]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2021-11-15 17:00:58,200] A new study created in memory with name: no-name-8a0377c6-a7f1-48ee-b0f9-849faec447ae
2021-11-15 17:00:59,044 postprocessor.py     58   [INFO] [__call__] start postprocessing 


  0%|          | 0/1114 [00:00<?, ?it/s]

[I 2021-11-15 17:01:05,879] Trial 0 finished with value: 0.7147498135716273 and parameters: {'e048': 0.5502929832141426, 'e049': 0.43168291616513543, 'e051': 0.6044204888004738, 'e053': 0.7334008903474662, 'e054': 0.8556197667070702, 'e074': 0.4977937664705141}. Best is trial 0 with value: 0.7147498135716273.
2021-11-15 17:01:06,695 postprocessor.py     58   [INFO] [__call__] start postprocessing 


  0%|          | 0/1114 [00:00<?, ?it/s]

[I 2021-11-15 17:01:13,427] Trial 1 finished with value: 0.7138365063522158 and parameters: {'e048': 0.6070504569936518, 'e049': 0.9416940983046302, 'e051': 0.24135888953063145, 'e053': 0.35608821803835267, 'e054': 0.9229751963375382, 'e074': 0.30650771572774127}. Best is trial 0 with value: 0.7147498135716273.
2021-11-15 17:01:14,259 postprocessor.py     58   [INFO] [__call__] start postprocessing 


...

[I 2021-11-15 17:13:24,537] Trial 97 finished with value: 0.7298106553829178 and parameters: {'e048': 0.19708211376245444, 'e049': 0.0008209099481518711, 'e051': 0.6470253082122985, 'e053': 0.9201805531317295, 'e054': 0.16842573621218168, 'e074': 0.957920712061726}. Best is trial 97 with value: 0.7298106553829178.
2021-11-15 17:13:25,406 postprocessor.py     58   [INFO] [__call__] start postprocessing 


  0%|          | 0/1114 [00:00<?, ?it/s]

[I 2021-11-15 17:13:32,211] Trial 98 finished with value: 0.719622145508591 and parameters: {'e048': 0.20422788648628284, 'e049': 0.004877338703420858, 'e051': 0.6526447996519481, 'e053': 0.9185413397420044, 'e054': 0.16964683855475546, 'e074': 0.6019033906176845}. Best is trial 97 with value: 0.7298106553829178.
2021-11-15 17:13:33,071 postprocessor.py     58   [INFO] [__call__] start postprocessing 


  0%|          | 0/1114 [00:00<?, ?it/s]

[I 2021-11-15 17:13:39,903] Trial 99 finished with value: 0.7286736116965024 and parameters: {'e048': 0.23200422298375428, 'e049': 0.039404220746285405, 'e051': 0.7180738643799847, 'e053': 0.9596028836014056, 'e054': 0.1263126651568041, 'e074': 0.9733007465605855}. Best is trial 97 with value: 0.7298106553829178.


In [20]:
d=study.best_params 
v=study.best_value
print(v, d)

0.7298106553829178 {'e048': 0.19708211376245444, 'e049': 0.0008209099481518711, 'e051': 0.6470253082122985, 'e053': 0.9201805531317295, 'e054': 0.16842573621218168, 'e074': 0.957920712061726}
